#Natural disasters, 2015-2019

Sources:
- https://www.ncei.noaa.gov/access/billions/events.pdf
- https://www.ncei.noaa.gov/access/billions/events/US/1980-2017?disasters[]=all-disasters
- https://www.noaa.gov/news/us-saw-10-billion-dollar-disasters-in-2015


## Setup environment

In [0]:
blob_container = "261storagecontainer"  
storage_account = "261storage" 
secret_scope = "261_team_6_1_spring24_scope"  
secret_key = "team_6_1_key"  
team_blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net" 


# blob storage is mounted here.
mids261_mount_path = "/mnt/mids-w261"

# SAS Token: Grant the team limited access to Azure Storage resources
spark.conf.set(
    f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
    dbutils.secrets.get(scope=secret_scope, key=secret_key),
)

# see what's in the blob storage root folder
# display(dbutils.fs.ls(f"{team_blob_url}"))

# mount
data_BASE_DIR = "dbfs:/mnt/mids-w261/"
# display(dbutils.fs.ls(f"{data_BASE_DIR}"))


## Import libraries

In [0]:
#standard
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F 
import seaborn as sns

# Boolean flags for sanity checks
from pyspark.sql.types import BooleanType, ArrayType


## Load file
- Uploaded .csv file from Google Drive to our blob via File --> Add data
- Queried using SQL
- Saved as Sparkdf
- Saved as Parquet

In [0]:
%sql
SELECT * FROM `hive_metastore`.`default`.`natural_disasters_2015_2019_to_df_version`;

In [0]:
natural_disasters_2015_2019_to_df_version = _sqldf
natural_disasters_2015_2019_to_df_version.display()

## Sanity checks
1. Check that all states are in list format
2. Check that there are no repeated states in any list
3. Check that there are no typos in states, based on list of all US states


### 1. Check that all states are in list format

In [0]:
#1. Check that all states are in list format********************************************

def is_list_of_strings(string):
    import ast
    try:
        values = ast.literal_eval(string)
        return isinstance(values, list) and all(isinstance(v, str) for v in values)
    except (ValueError, SyntaxError):
        return False

is_list_of_strings_udf = spark.udf.register("is_list_of_strings", is_list_of_strings)

# Trim leading/trailing whitespace and remove single quotes
cleaned_states = natural_disasters_2015_2019_to_df_version.withColumn("states", F.trim(F.regexp_replace(F.col("states"), "'", "")))

# Check if each string represents a list of strings
result = cleaned_states.select("*", is_list_of_strings_udf(F.col("states")).alias("is_list_of_strings"))
invalid_rows = result.where("is_list_of_strings = false").select("*").collect()

if len(invalid_rows) == 0:
    print("The 'states' column contains strings representing lists of strings.")
else:
    print("The following rows do not contain strings representing lists of strings:")
    for row in invalid_rows:
        print(row)

In [0]:
cleaned_states.display()

### 2. Check that there are no repeated states in any list

In [0]:
#create id column for disasters
first_three_letters = F.lower(F.substring(F.col("disaster_name"), 1, 3))
start_date_without_hyphens = F.regexp_replace(F.col("start_date"), "-", "")
end_date_without_hyphens = F.regexp_replace(F.col("end_date"), "-", "")
disaster_id = F.concat(first_three_letters, start_date_without_hyphens,end_date_without_hyphens)
cleaned_states = cleaned_states.withColumn("disaster_id", disaster_id)
cleaned_states.display()

In [0]:
# check that all ids are unique. 
unique_disaster_ids = cleaned_states.select("disaster_id").distinct()
num_unique_disaster_ids = unique_disaster_ids.count()

num_rows_cleaned_states = cleaned_states.count()

if num_unique_disaster_ids == num_rows_cleaned_states:
    print("The number of unique disaster_ids is equal to the number of rows in cleaned_states.")
else:
    print("The number of unique disaster_ids is not equal to the number of rows in cleaned_states.")

In [0]:
# Check that there are no repeated states*****************************************************

# Split the states column into an array of individual states
split_states = cleaned_states.selectExpr("disaster_id", "split(states, ',') as states_array")

# Explode the array to create one row per state
exploded_states = split_states.selectExpr("disaster_id", "explode(states_array) as state")

# Count the occurrences of each state
state_counts = exploded_states.groupBy("disaster_id", "state").count()

# Check if any state occurs more than once within a row
invalid_states = state_counts.where("count > 1")

if invalid_states.count() == 0:
    print("No repeated two-letter combinations within each row.")
else:
    print("The following rows contain repeated two-letter combinations:")
    invalid_states.show()

### 3. Check that there are no typos in states, based on list of all US states

In [0]:
#50 states, plus PR and US Virgin Islands
valid_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'VI', 'PR']

@udf(BooleanType())
def is_valid_state(state):
    return state in valid_states

@udf(BooleanType())
def validate_states(states_str):
    states = states_str.strip('[]').split(',')
    return all(state.strip() in valid_states for state in states)

# Apply the UDF to the states column
df = cleaned_states.withColumn("is_valid", validate_states(F.col("states")))

# Show the DataFrame with the validation result
df.display()

RA in disaster_id nor2017022820170301 is wrong from the base documents (both https://www.ncei.noaa.gov/access/billions/events.pdf and https://www.ncei.noaa.gov/access/billions/events/US/2015-2018?disasters[]=all-disasters), so will be removed. 

In [0]:
# Remove 'RA' from the states column for disaster_id nor2017022820170301
cleaned_states = cleaned_states.withColumn("states", F.regexp_replace("states", "RA,", ""))
cleaned_states.display()


## Split states and dates into rows


In [0]:
# Remove square brackets from states column in cleaned_states
exploded_states = cleaned_states.withColumn("states", F.regexp_replace("states", r"\[\[\]\]", ""))

# Use explode to split states in cleaned_states into individual rows, leaving one value per row
exploded_states = exploded_states.select("*", F.explode(F.split(exploded_states.states, ",")).alias("state"))

# remove extra bracket
exploded_states = exploded_states.withColumn("state", F.regexp_replace("state", r"[\[\]]", ""))

# Show the exploded states DataFrame
exploded_states.display()

In [0]:
# Generate a new dataframe with one row per date between start_date and end_date for each state
exploded_dates = exploded_states.select("*", F.explode(F.expr("sequence(to_date(start_date), to_date(end_date), interval 1 day)")).alias("date"))

# Show the exploded dates DataFrame
exploded_dates.display()

In [0]:
# lowercase all values in the disaster_type column
exploded_dates = exploded_dates.withColumn("disaster_type", F.lower(F.col("disaster_type")))
exploded_dates = exploded_dates.withColumn("disaster_type", F.regexp_replace(F.col("disaster_type"), " ", "_"))
exploded_dates.display()

In [0]:
natural_disasters_2015_2019 = exploded_dates.select("disaster_name", "disaster_type", "state", "date")
natural_disasters_2015_2019.display()

## Save as Parquet file

In [0]:
# save as parquet file
natural_disasters_2015_2019.write.mode("overwrite").parquet(f"{team_blob_url}/5y_natural_disasters")

In [0]:
# Load checkpointed file
natural_disasters_2015_2019 = spark.read.parquet( f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net/5y_natural_disasters" )